### This file carries out orthogonal projection from a Cartesian point in space to an ellipsoid Earth.

Equatorial Radius $r_{e} =  6,378,136.6 $ $ (m)$

Polar Radius $r_{p} =   6,356,751.9 $ $ (m)$


Mathematical Expression of Earth in Cartesian coordinates:

$$ \frac{x^2}{{r_e}^2} + \frac{y^2}{{r_p}^2} + \frac{z^2}{{r_e}^2} = 1 $$


In [ ]:
import numpy as np

class Earth():
    def __init__(self):
        self.re = 6378136.6
        self.rp = 6356751.9
